<center>

# Riemannian L-systems: Modeling growing forms in curved spaces

## Christophe Godin<sup>1</sup> and Frédéric Boudon<sup>2,3</sup>
</center>


<center>
<sup>1</sup>Laboratoire Reproduction et Développement des Plantes, Univ Lyon, ENS de Lyon, UCB Lyon1, CNRS,
INRAE, Inria, F-69342 Lyon, France
</center>

<center>
<sup>2</sup>CIRAD, UMR AGAP Institute, F-34398 Montpellier, France
</center>
<center>
<sup>3</sup>UMR AGAP Institute, Univ. Montpellier, CIRAD, INRAE, Institute Agro, Montpellier, France
</center>

<center>
christophe.godin@inria.fr, frederic.boudon@cirad.fr
</center>

## Abstract

In the past 50 years, L-systems have been successfully used to model the development of filamentous and branching forms in biology. Their success is largely due to the fact that they rely on the mathematical notion of discrete rewriting systems, that in essence simply reflects the idea that the evolution of a structure results from the evolution of its individual components. This core property is reminiscent of how biological organisms develop and happens to be critical to model their growth. The formalism of L-systems has been developed to model the growth of forms in Euclidean 1-D, 2-D, or 3-D spaces. These spaces have the property to be flat and show no curvature anywhere. However, the growth of various forms or processes in biology takes place in curved spaces. This is for example the case of vein networks growing within curved leaf blades, of unicellular tubes, such as pollen tubes, growing on curved surfaces to fertilize distant ovules, of teeth patterns growing on folded epithelia of animals, of diffusion of chemical or mechanical signals at the surface of plant or animal tissues, etc. To model these growing forms in curved spaces, we thus developed further the theory of L-systems. In a first step we show that this extension can be carried out by integrating concepts of differential geometry in the notion of turtle geometry. We then illustrate how this extension can be applied to model and program the development of both mathematical and biological forms on curved surfaces embedded in our Euclidean space. We provide various examples applied to plant development. We finally show that this approach can be extended to more abstract spaces, called abstract Riemannian spaces, that are not embedded into any higher space, while being intrinsically curved. We suggest that this abstract extension can be used to provide a new approach for effective modeling of tropism phenomena and illustrate this idea on a few conceptual examples.

In [7]:
from pgljupyter import *
from IPython.display import Code, display

def display_example(filename, caption, size_world=50, animate=False):
      print('# '+caption)
      display(Code(filename='../Lsystems/'+filename, language='python'))
      display(LsystemWidget('../Lsystems/'+filename, size_world=size_world, animate=animate))

def display_examples(examples, size_world=50, animate=False):
    for filename, caption in examples:
        display_example(filename, caption, size_world=size_world, animate=animate)


## 2. L-systems overview

In [8]:
examples = [('Fig01-Listing1.lpy','Listing 1: Archimedean spiral'),
            ('Fig01-Listing2.lpy','Listing 2: Random walk in 2-D')]

display_examples(examples)

# Listing 1: Archimedean spiral


Axiom: A(1)
derivation length: 20
production:
A(n) --> F(n)+(60)A(n+1)
endlsystem

LsystemWidget(derivationLength=20, derivationNumber=None, scene={'data': b'x\xdaSLrw\xf5\xf7e`Pp\xe0\xe5RPVVd\…

# Listing 2: Random walk in 2-D


from random import random
dl = 1.
Axiom : A (0)
derivation length : 100
production :
A(n) :
  a = 360* random()
  nproduce +(a) F(dl) A(n+1)

LsystemWidget(derivationLength=100, derivationNumber=None, scene={'data': b'x\xdaSLrw\xf5\xf7e`Pp\xe0\xe5RPVVd…

In [10]:
examples = [('Fig01-Listing3.lpy','Listing 3: Fractal curve (von Koch flake) '),
            ('Fig01-Listing4.lpy','Listing 4: Plant branching structure development')]

display_examples(examples, animate=True)

# Listing 3: Fractal curve (von Koch flake) 


Axiom : _(0.001)F(1) -(120) F(1) -(120) F(1)
derivation length : 5
production :
F(x) : nproduce F(x/3.0) + F(x/3.0) -- F(x/3.0) + F(x/3.0)
endlsystem

LsystemWidget(animate=True, derivationLength=5, derivationNumber=None, scene={'data': b'x\xdaSLrw\xf5\xf7e`Pp\…

# Listing 4: Plant branching structure development


N = 10                                
offset = 2                                   
iangle = 60

Axiom:  _(0.5)A(0)
derivation length: N
production: 
A(n) :
  if n<N:      # produces an internode, a lateral bud (in [...]) and an apical bud
    nproduce I(n) [P(n)A(n+offset)] A(n+1)
  else:        # produces a flower bud
    nproduce B 

interpretation:
I(n) : nproduce ;(1)F(N-n)
P(n) :         # Phyllotaxis angle
  if n%2 == 0 : nproduce  +(iangle)
  else: nproduce -(iangle)
A(n): nproduce ;(2)@O(1.5)
B : nproduce F(1);(3)@O(1.5)

endlsystem

LsystemWidget(animate=True, derivationLength=10, derivationNumber=None, scene={'data': b"x\xdaSLrw\xf5\xf7e`Pp…

## 3. Moving on parametric surfaces

In [13]:
display_example('Fig08-Listing5.lpy','Figure 8: Holonomy and parallel transport using Riemannian Lsystems}. (a-b) parallel transport of a vector along a polygon made of geodesics. The vector is initially tangent to the first geodesic, then is perpendicular to the tangent on the second geodesic, then points backward on the third geodesic segment. (d-c) parallel transport of a vector not tangent to the first geodesic. The vector keeps a constant angle with the tangent vector, but this angle changes each time the turtle turns. (e-f) parallel transport along a curve that is not a geodesic: the angle between the transporting curve and the transported vector varies continuously.')

# Figure 8: Holonomy and parallel transport using Riemannian Lsystems}. (a-b) parallel transport of a vector along a polygon made of geodesics. The vector is initially tangent to the first geodesic, then is perpendicular to the tangent on the second geodesic, then points backward on the third geodesic segment. (d-c) parallel transport of a vector not tangent to the first geodesic. The vector keeps a constant angle with the tangent vector, but this angle changes each time the turtle turns. (e-f) parallel transport along a curve that is not a geodesic: the angle between the transporting curve and the transported vector varies continuously.


from riemann_lsystem.surfaces import *
from riemann_lsystem.riemannianturtle import *
from math import pi

%pastefile ../src/riemann_lsystem/riemannianrules.lpy

R = 1
alpha0 = -30     # in degrees
Axiom: 
  nproduce SetSpace(Sphere(R)) _(0.01)
  nproduce [,(0) PlotSpace() ]
  nproduce InitTurtle((0,0,0,1))
  nproduce ParallelTransportReset
  nproduce [,(3) ParallelTransportedArrow(alpha0,0.5) ]
  nproduce Path(R*pi/2) 
  nproduce +(90) Path(R*pi/2) 
  nproduce +(90) Path(R*pi/2) 

derivation length:1
production:
interpretation:
Path(l):
  for i in range(10):
    nproduce F(l/10) [,(3) ParallelTransportedArrow(alpha0,0.5) ]
endlsystem
###### INITIALISATION ######

__lpy_code_version__ = 1.1

def __initialiseContext__(context):
	import openalea.plantgl.all as pgl
	Color_14 = pgl.Material("Color_14" , transparency = 0.45 , )
	Color_14.name = "Color_14"
	context.turtle.setMaterial(14,Color_14)
	context.animation_timestep = 0.001
	scalars = [('a', 'Float', 1.0, 0.1, 5.0, 2), ('b', 'Float', 2.87, 0.1, 5.0, 2)]
	context["__scalars__"] = scalars
	for s in scalars:
		if not s[1] == "Category" : context[s[0]] = s[2]
	import openalea.plantgl.all as pgl
	pinfunc = pgl.NurbsCurve2D(	
	    ctrlPointList = pgl.Point3Array([(0, 0.05, 1),(0.0606061, 0.663636, 1),(0.329545, 0.213636, 1),(0.725758, 0.131818, 1),(1, 0.118182, 1)]) , 
	    )
	pinfunc.name = "pinfunc"
	panel_0 = ({'name': 'Panel 2', 'active': True, 'visible': False},[('Function',pinfunc)])
	parameterset = [panel_0,]
	context["__functions__"] = [('pinfunc',pinfunc),]
	context["__curves__"] = []
	context["__parameterset__"] = parameterset
	context["pinfunc"] = pgl.QuantisedFunction(pinfunc)

Fig08-Listing5.lpy:375: Warning: IndentationWarning: missing indent.
Fig08-Listing5.lpy:379: Warning: IndentationWarning: missing indent.
Fig08-Listing5.lpy:382: Warning: IndentationWarning: missing indent.
Fig08-Listing5.lpy:387: Warning: IndentationWarning: missing indent.
Fig08-Listing5.lpy:390: Warning: IndentationWarning: missing indent.
Fig08-Listing5.lpy:419: Warning: IndentationWarning: missing indent.
Fig08-Listing5.lpy:433: Warning: IndentationWarning: missing indent.
Fig08-Listing5.lpy:475: Warning: IndentationWarning: missing indent.
Fig08-Listing5.lpy:486: Warning: IndentationWarning: missing indent.
Fig08-Listing5.lpy:513: Warning: IndentationWarning: missing indent.
Fig08-Listing5.lpy:526: Warning: IndentationWarning: missing indent.
Fig08-Listing5.lpy:540: Warning: IndentationWarning: missing indent.
Fig08-Listing5.lpy:542: Warning: IndentationWarning: missing indent.
Fig08-Listing5.lpy:550: Warning: IndentationWarning: missing indent.
Fig08-Listing5.lpy:552: Warning: I

LsystemWidget(derivationLength=1, derivationNumber=None, scene={'data': b'x\xdaL\xdd\x05TV\xcb\x16\x07p\x10\x1…